In [1]:
!pip install git+https://github.com/PyFstat/PyFstat@python37

  Cloning https://github.com/PyFstat/PyFstat (to revision python37) to /tmp/pip-req-build-1x0ekb1r
  Running command git clone --filter=blob:none --quiet https://github.com/PyFstat/PyFstat /tmp/pip-req-build-1x0ekb1r
  Running command git checkout -b python37 --track origin/python37
  Switched to a new branch 'python37'
  Branch 'python37' set up to track remote branch 'python37' from 'origin'.
  Resolved https://github.com/PyFstat/PyFstat to commit 73ad1acdc9385a234727abf3eb9f93c9298fc5e5
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 575.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.9 MB/s eta 0:00:00
  Installing build dependencies ... - \ |

In [2]:
import os, sys, shutil, gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyfstat
from scipy import stats
from scipy import signal
from scipy.interpolate import interp1d
from scipy.signal import butter, filtfilt, iirdesign, zpk2tf, freqz
import seaborn as sns
import h5py
import joblib
%matplotlib inline

In [3]:
shape_1 = (180,23)
shape_2 = (360,1)

In [4]:
class data_gen():
    def __init__(self, signal = False):
        self.writer_kwargs = {
                        "tstart": 1238166018,
                        "detectors": "H1,L1",   
                        "SFTWindowType": "tukey", 
                        "SFTWindowBeta": 0.01,
                        "Tsft": 1800,
                        "duration": 1800*4583.0,
                       }
        
        # TODO: ここ調整！！！
        # 1/5.01 -> 0.1996007984031936
        self.priors={
              'Band': 1/5.01,
          }
#         self.priors={
#             'Band': 0.36,
#         }
#         self.priors={
#             'Band': 0.3,
#         }
#         self.priors={
#             'Band': 0.36,
#         }

        self.random_params = {
            "F0": lambda: stats.uniform(50,500).rvs(),
            "F1": lambda: 10**stats.uniform(-12, 4).rvs(),
            "cosi": lambda: stats.uniform(-1.0,1.0).rvs(),
            'psi': lambda: stats.uniform(-0.25 * np.pi, 0.25 * np.pi).rvs(),
            'phi': lambda: stats.uniform(0,2*np.pi).rvs(),
            'Alpha': lambda: stats.uniform(0,3.14159).rvs(),
            'Delta': lambda: stats.uniform(0,3.14159).rvs(),
        }
        if signal:
            # TODO: ここ調整！！！
            #self.random_params["h0"] = lambda: 0.5e-23 / stats.uniform(1, 10).rvs()
            # self.random_params["h0"] = lambda: 1e-23 / stats.uniform(1, 6).rvs()
            # self.random_params["h0"] = lambda: 1e-23 / stats.uniform(2, 10).rvs()
            
            # self.random_params["h0"] = lambda: 1e-23 / stats.uniform(1, 7).rvs()
            # self.random_params["h0"] = lambda: 1e-23 / stats.uniform(1, 8).rvs()
            self.random_params["h0"] = lambda: 1e-23 / stats.uniform(1, 5).rvs()
        else:
            self.writer_kwargs["sqrtSX"]= 0.5e-23
    def __len__(self):
        return 100
    def __getitem__(self, idx):
        try: 
            target_params = {name:self.random_params[name]() for name in self.random_params.keys()}
            priors = self.priors
            priors.update(target_params)
            signal_parameters_generator = pyfstat.AllSkyInjectionParametersGenerator(
                priors=priors
            )
            params = signal_parameters_generator.draw()
            self.writer_kwargs["outdir"] = f"PyFstat_example_data_ensemble/Signal"
            self.writer_kwargs["label"] = f"Signal"

            writer = pyfstat.Writer( **self.writer_kwargs, **params)
            data = []
            writer.make_data()
            frequency, timestamps, amplitudes = pyfstat.utils.get_sft_as_arrays(
                writer.sftfilepath
            )

        except Exception as e:
#             print(e)
            return self[idx]
        frequency=frequency[:360]
        for key in timestamps.keys():
            amplitudes[key] = amplitudes[key][:360,:shape_1[0]*shape_1[1]].astype(np.complex128)
            amplitudes[key] = amplitudes[key] * 1e22
            amplitudes[key] = amplitudes[key].real**2 + amplitudes[key].imag**2
                                                                                  
            amplitudes[key] = np.mean(amplitudes[key].reshape(360, shape_1[0],shape_1[1]), axis=2)
            amplitudes[key] = np.mean(amplitudes[key].reshape(shape_2[0],shape_2[1],shape_1[0]), axis=1).astype(np.float16)
            timestamps[key] = np.mean(timestamps[key][:shape_1[0]*shape_1[1]].reshape(shape_1), axis=1)
            gc.collect()
        frequency = np.mean(frequency.reshape(shape_2),axis=1)
        return frequency, timestamps, amplitudes

Generate Raw Signals

In [5]:
gen = data_gen(signal=1)
amps = []
out_dir = '1_data'
try:
    os.mkdir(out_dir)
except:
    pass

positive_data_len = 2000
# positive_data_len = 3

# for i in range(1000):
# 0~999
for i in range(positive_data_len):
    frequency, timestamps, amp= gen[i]
    gc.collect()
    np.save(f'./{out_dir}/signals_{i}', arr=np.stack([amp[key] for key in amp.keys()]))
    
    
#     np.save(f'./{out_dir}/signals_{i%1000}', arr=np.stack([amp[key] for key in amp.keys()]))
#     if i%100 == 99:
#         shutil.rmtree(f'./PyFstat_example_data_ensemble')
#     if i%1000 == 999:
#         shutil.make_archive(f'./{out_dir}_{i+1}', 'zip', f'./{out_dir}/')
#         shutil.rmtree(f'./{out_dir}')
#         os.mkdir(f'./{out_dir}')

Generate Noise

In [6]:
gen = data_gen(signal=0)
amps = []
out_dir = '0_data'
try:
    os.mkdir(out_dir)
except:
    pass

negative_data_len = 2000
# negative_data_len = 3

# for i in range(1000):
for i in range(negative_data_len):
    frequency, timestamps, amp = gen[i]
    gc.collect()
    # positive_data_len + i -> 1000から1999まで
    np.save(f'./{out_dir}/signals_{positive_data_len + i}', arr=np.stack([amp[key] for key in amp.keys()]))

#     np.save(f'./{out_dir}/signals_{i%1000}', arr=np.stack([amp[key] for key in amp.keys()]))
#     if i%100 == 99:
#         shutil.rmtree(f'./PyFstat_example_data_ensemble')
#     if i%1000 == 999:
#         shutil.make_archive(f'./{out_dir}_{i+1}', 'zip', f'./{out_dir}/')
#         shutil.rmtree(f'./{out_dir}')
#         os.mkdir(f'./{out_dir}')

In [7]:
print("完了しました！")

完了しました！
